# Analyzing borrowers’ risk of defaulting

In this project we'll prepare a report for a bank’s loan division. We’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The report will be based on data collected by the bank on customers’ credit worthiness.

The report will be considered when building a credit scoring of a potential customer. We'll start with preprocessing the data including filling missing values, removing duplicates, changing data types and categorizing the data so we can make worthy and valueble conclusions.

##  Data file and general info

In [1]:
import pandas as pd
import nltk
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

#open the data file
try:
    df = pd.read_csv('credit_scoring_eng.csv')
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')

#show general information
display(df.info(),df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [2]:
#show additional information
display(
    [df['children'].unique(),df['dob_years'].unique(), df['education'].unique(), 
     df['family_status'].unique(), df['gender'].unique(), df['income_type'].unique(), df['purpose'].unique(), ]
       )

[array([ 1,  0,  3,  2, -1,  4, 20,  5]),
 array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
        21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
        64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
        66, 69, 19, 72, 70, 74, 75]),
 array(["bachelor's degree", 'secondary education', 'Secondary Education',
        'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
        'primary education', "Bachelor's Degree", 'SOME COLLEGE',
        'Some College', 'PRIMARY EDUCATION', 'Primary Education',
        'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
       dtype=object),
 array(['married', 'civil partnership', 'widow / widower', 'divorced',
        'unmarried'], dtype=object),
 array(['F', 'M', 'XNA'], dtype=object),
 array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
        'entrepreneur', 'student', 'paternity / maternity leave'],
       dtype=object),
 array(['purchase

### Conclusion

- There are missing values in `days_employed` and `total_income` columns. They have the same amount of records, so it's better to check if it's related to each other.
- A more convinient way to call the column `dob_years`, would probably be `age`.
- Ask the data engineers how and from where the information about `days_employed` gathered, becuase it doesn't make any sense.
- Maybe we can reduce memory usage by changing the int64, to int32.
- The `education` values have duplicates, thus they should be standardized, lowercase preffered.
- The `children` column has some negative or extreme values that doesn't make sense. Also the `dob_years` column have some values of 0 which is probably wrong.
- The `purpose` column should become standardized to simple categories by stemming and lemmatizing.
- Maybe the `total_income` should be changed to int, because in such big numbers the floating point is irelevant.
- The `debt` coulmn should be boolean.
- The `gender` column has a 'XNA' value which probably means the standart gender value was missing, we'll think how to deal with it based on other information.

## Data preprocessing

### Processing missing values

First we will save a copy of the original data frame and change the `dob_years` to `age`. Than we'll check on which columns the missing values are.

In [3]:
#save a copy of the original dataset
df_original = df.copy()
#replace the 'dob_years' column name to 'age'
column_name = list(df.columns)
column_name[2] = 'age'
df.set_axis(column_name,axis = 'columns',inplace = True)
#check if changed
df.columns

Index(['children', 'days_employed', 'age', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Now we'll check for the 'XNA' value in the `gender` column.   We'll check the amount and decide how to act.

In [4]:
df[df['gender'] == 'XNA']

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


There is only one record, so it will be better to remove it than guessing the gender.

In [5]:
#remove the row that contains 'XNA' value
df = df[df['gender'] != 'XNA']
#check the change
df['gender'].unique()

array(['F', 'M'], dtype=object)

No more 'XNA' values.

There are also some negative values in `children` and `days_employed` which don't make sense. We will change them at first to positive numbers.

In [6]:
#make all values positive
df['days_employed'] = abs(df['days_employed'])
df['children'] = abs(df['children'])

The column `dob_years` has been changed successfully to`age`, there are only two genders categories and all values are positive!
Now for the missing values:

In [7]:
#check the misiing values by columns, show in percentage
df.isna().sum()*100/len(df)

children             0.000000
days_employed       10.100353
age                  0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.100353
purpose              0.000000
dtype: float64

In [8]:
#check if all the missing values related to each other
len(df[df['days_employed'].isnull() & df['total_income'].isnull()])

2174

As we can see, the values of `'total_income` and `days_employed` *missing both at the same rows*.
Probably they related to each other, maybe calculated one from another. The percentage of the missing values is around 10% of all the records. Now we will check if there's a difference between the genders.

In [9]:
#check if the missing values related to a gender
missing_by_gender = df[df['days_employed'].isnull()]['gender'].value_counts()
missing_by_gender

F    1484
M     690
Name: gender, dtype: int64

Women's records missing values are more then twice then men's. We can try to evaluate the total income, based on other parameters too. The total income is a quantitative variables so we can try to fill it with median or mean.
We'll check if there's a differnce in the total income based on education and the gender to decide how to fill the missing values. Also we'll check why more women lack that information. We won't change the `days_employed` values because we don't need them for our analyzings right now.
First we will check what is the percentage of the missing values by gender:

In [10]:
#totals of each gender
totals_gender = df['gender'].value_counts()

#calculate the precentage of the missing values by gender
women_precentage = missing_by_gender[0]/totals_gender[0]
print('The precentage of women missing values: {:.2%}'.format(women_precentage))
men_precentage = missing_by_gender[1]/totals_gender[1]
print('The precentage of man missing values: {:.2%}'.format(men_precentage))

The precentage of women missing values: 10.42%
The precentage of man missing values: 9.47%


It doesn't seem to tend by gender, it has almost the same percantge as the missing values in total.
Let's check if there are other tendencies 

In [11]:
#check if the missing values related to the type of income
missing_by_income = df[df['days_employed'].isnull()]['income_type'].value_counts()
totals_income = df['income_type'].value_counts()
#print(totals_income) for knowing the right index
#print()
print('The precentage of employees missing values: {:.2%}'.format(missing_by_income[0]/totals_income[0]))
print('The precentage of businesses missing values: {:.2%}'.format(missing_by_income[1]/totals_income[1]))
print('The precentage of retirees missing values: {:.2%}'.format(missing_by_income[2]/totals_income[2]))
print('The precentage of civil servants missing values: {:.2%}'.format(missing_by_income[3]/totals_income[3]))
print('The precentage of entrepreneur missing values: {:.2%}'.format(missing_by_income[4]/totals_income[4]))

The precentage of employees missing values: 9.94%
The precentage of businesses missing values: 9.99%
The precentage of retirees missing values: 10.71%
The precentage of civil servants missing values: 10.08%
The precentage of entrepreneur missing values: 50.00%


Again we see that the values are about the same, except the entrepreneur. Let's check!

In [12]:
df[df['income_type'] == 'entrepreneur']['gender'].value_counts()

M    1
F    1
Name: gender, dtype: int64

We have only two records so the missing values are not related to income type.

In [13]:
missing_by_status = df[df['days_employed'].isnull()]['family_status'].value_counts()
missing_by_status
totals_status = df['family_status'].value_counts()
#print(totals_status) for knowing the right index
#print()
print('The precentage of married missing values: {:.2%}'.format(missing_by_status[0]/totals_status[0]))
print('The precentage of civil partnership missing values: {:.2%}'.format(missing_by_status[1]/totals_status[1]))
print('The precentage of unmarried missing values: {:.2%}'.format(missing_by_status[2]/totals_status[2]))
print('The precentage of divorced missing values: {:.2%}'.format(missing_by_status[3]/totals_status[3]))
print('The precentage of widow / widower missing values: {:.2%}'.format(missing_by_status[4]/totals_status[4]))

The precentage of married missing values: 9.99%
The precentage of civil partnership missing values: 10.58%
The precentage of unmarried missing values: 10.24%
The precentage of divorced missing values: 9.37%
The precentage of widow / widower missing values: 9.90%


It seems like there is no clear tendecy, and the percentage still the same. We'll try to think of a strategy for filling the missing values. First we'll try to fill the total income column. Let's see if the income depend on other parameters. Usually the education and the type of income affect the income, also the gender...

In [14]:
#check the difference in the total income based on education
#first standardize the education column so it will be easier to analayz
df['education'] = df['education'].str.lower()
df.groupby('education').agg({'total_income':['mean', 'median']})

total_income            
                             mean      median
education                                    
bachelor's degree    33142.802434  28054.5310
graduate degree      27960.024667  25161.5835
primary education    21144.882211  18741.9760
secondary education  24594.503037  21836.5830
some college         29040.132990  25608.7945

We can see there are differences in the total income, based on the eduction. Now we'll check if it also depends on income type and gender.

In [15]:
#check the difference in the total income based on income type and gender
df.groupby(['income_type','gender']).agg({'total_income':['mean', 'median']})

total_income            
                                            mean      median
income_type                 gender                          
business                    F       29475.489877  25731.3245
                            M       37284.334474  31491.0080
civil servant               F       24908.849830  21917.1980
                            M       34036.170503  29754.3915
employee                    F       23818.105441  20898.4980
                            M       28956.563225  25945.7880
entrepreneur                F       79866.103000  79866.1030
                            M                NaN         NaN
paternity / maternity leave F        8612.661000   8612.6610
retiree                     F       21446.167378  18529.2465
                            M       24117.476626  20918.3620
student                     M       15712.260000  15712.2600
unemployed                  F       32435.602000  32435.6020
                            M        9593.119000   9593.1190

As we can see, in all categories except the **'unemployed'** and **'maternity leave'**, men earn more money. We don't know about **'entrepreneur'** because there is not enough information. Also the total income veries between the different income types, where the income types of people that are being well paid are entrepreneur, unemployed (for women) and business. In conclusion, the total income depend on gender, education and type of income, thus we will fill the missing values, based on these parameters. In some categories the mean and median are slightly different, so median is prefferd. Because there is only one man as an entrepreneur, and there is not enough information to base on for calculating the median, we'll delete this record. The `days_employed` missing values won't be changed because they are irrelevant for now and we don't know the right way to do so.

In [16]:
df['total_income'].fillna(df.groupby(['gender','education','income_type'])['total_income'].transform('median'), inplace = True)
#check that missing values were filled and the only male entrepreneur left           
display(df['total_income'].isna().sum(), df[df['total_income'].isnull()]['income_type'])
#delete the last row with missing value
display(df.dropna(subset = ['total_income'], inplace = True), df.isnull().sum())

1

5936    entrepreneur
Name: income_type, dtype: object

None

children               0
days_employed       2173
age                    0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

The only missing values are on the `days_employed` column but we left it like this intentionally.

#### Conclusion

10% of the rows were missing the values of `days_employed` and `toal_income`. The data in `days_employed` and `toal_income` based on each other, because we don't know how, it's better to try to fill the data we can find. The days employed are not for sure relevant for our research, but the total income is, so based on selected parameters we calculated the median based on those parameters. Also the `days_employed` column has a lot of wierd values, but as we mentioned, we considered it as irrelevant and decided to leave it untouched. 

### Data type replacement

The `total_income` column should be changed to int because the floating point is giving extra irrelevant information and it makes it clean and easy to analayz. The `days_employed` won't be changed, there are still missing values and we don't use it for our calculations. We'll change the type of `debt` column to boolean, because it makes more sense, and all the ids cloumns we'll change to less memory consumable int type (the original memory usage is very low, but we'll do it for practice). Also there were negative values of children and an extreme one as 20, we assume these are typos so we already converted negative values to positive, and now we will change the 20 to 2.

In [17]:
#change data types
df = df.astype(
    {'age':'int8','children':'int32', 'education_id':'int8','family_status_id':'int8' ,'total_income':'int','debt':'bool'})
#convert the 20 to 2
df.loc[df['children'] == 20,'children'] = 2
#check if changed
display(df.info(), df['children'].unique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21523 entries, 0 to 21524
Data columns (total 12 columns):
children            21523 non-null int32
days_employed       19350 non-null float64
age                 21523 non-null int8
education           21523 non-null object
education_id        21523 non-null int8
family_status       21523 non-null object
family_status_id    21523 non-null int8
gender              21523 non-null object
income_type         21523 non-null object
debt                21523 non-null bool
total_income        21523 non-null int64
purpose             21523 non-null object
dtypes: bool(1), float64(1), int32(1), int64(1), int8(3), object(5)
memory usage: 1.5+ MB


None

array([1, 0, 3, 2, 4, 5])

#### Conclusion

As we can see, the relevant columns types have been changed and also the memory usage lowered from 2MB to 1.3MB. Because the change is between numeric types, we used the astype() method.

### Processing duplicates

At first, let's see amount of duplicates and display a small amount of data to see what we can infer from it.

In [18]:
display(df.duplicated().sum(),df[df.duplicated()].head(), df[df.duplicated()].tail())

71

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,secondary education,1,married,0,F,employee,False,19860,purchase of the house for my family
3290,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,False,18046,to have a wedding
4182,1,NaN,34,bachelor's degree,0,civil partnership,1,F,employee,False,24590,wedding ceremony
4851,0,NaN,60,secondary education,1,civil partnership,1,F,retiree,False,18046,wedding ceremony
5557,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,False,18046,to have a wedding


,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20702,0,NaN,64,secondary education,1,married,0,F,retiree,False,18046,supplementary education
21032,0,NaN,60,secondary education,1,married,0,F,retiree,False,18046,to become educated
21132,0,NaN,47,secondary education,1,married,0,F,employee,False,19860,housing renovation
21281,1,NaN,30,bachelor's degree,0,married,0,F,employee,False,24590,buy commercial real estate
21415,0,NaN,54,secondary education,1,married,0,F,retiree,False,18046,housing transactions


We can see a lot of nan values in the `days_employed` column, let's check if all the duplicates are missing this value.

In [19]:
#check the amount of duplicates, where the days_employed is missing
df[df['days_employed'].isnull()].duplicated().sum()

71

All the duplicated rows are with nan values in the `days_employed` and `total_income` (before we changed the missing values in `total_income`). We will pass this information to the data engineers so they'l investigate it. We'll remove those duplicated rows and reset the indexes.

In [20]:
#drop the duplicated rows
df = df.drop_duplicates().reset_index(drop = True)
#check if it changed
df.duplicated().sum()

0

#### Conclusion

The duplicates found only in the rows where the `days_employed` and `total_income` where missing. It is better to inform the source who gave us the data so they'll explain or fix it, maybe there is a bug. The duplicated values have been removed.

### Categorizing Data

We need to categorize the `purpose` column to one or two simple words, the `total_income` to income levels, `children` to has or not and also will be helpful to categorize by `age`, so we can get the right conclusions. We'll start with categorizing the purpose. First we will find the unique purposes, tokenize them while ignoring stop words (a, the, to etc.). Then we'll lemmatize this group of words, stem them and find the most frequent and describeful for each category and save them in a unique list. Then a function will get the purpose of a specific record, and will stem the words and check if there's any compatibility to the categories list of words.

In [21]:
#convert the unique values of 'purpose' column to one long string
words = " ".join(df['purpose'].unique().astype('str'))
#tokenize the words of the string
tokenized = nltk.word_tokenize(words)
#ignore stop words (by using stopwords.words('english')) and duplicates (by using set())
filtered_words = set(list(filter(lambda word: word not in stopwords.words('english'), tokenized)))
#show the unique values of the 'purpose' column, the string we got and the filtered values 
display(df['purpose'].unique(),words,filtered_words)

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

'purchase of the house car purchase supplementary education to have a wedding housing transactions education having a wedding purchase of the house for my family buy real estate buy commercial real estate buy residential real estate construction of own property property building a property buying a second-hand car buying my own car transactions with commercial real estate building a real estate housing transactions with my real estate cars to become educated second-hand car purchase getting an education car wedding ceremony to get a supplementary education purchase of my own house real estate transactions getting higher education to own a car purchase of a car profile education university education buying property for renting out to buy a car housing renovation going to university'

{'become',
 'building',
 'buy',
 'buying',
 'car',
 'cars',
 'ceremony',
 'commercial',
 'construction',
 'educated',
 'education',
 'estate',
 'family',
 'get',
 'getting',
 'going',
 'higher',
 'house',
 'housing',
 'profile',
 'property',
 'purchase',
 'real',
 'renovation',
 'renting',
 'residential',
 'second-hand',
 'supplementary',
 'transactions',
 'university',
 'wedding'}

In [22]:
wordnet_lemma = WordNetLemmatizer()
#lemmatize the nouns in the list of the filtered words
lemmas = [wordnet_lemma.lemmatize(w, pos='n') for w in filtered_words]
#stem the lemmas and print how it changed
english_stemmer = SnowballStemmer('english')
for word in lemmas:
    stemmed_word = english_stemmer.stem(word)
    print(word,'-->',stemmed_word)

renovation --> renov
commercial --> commerci
university --> univers
going --> go
estate --> estat
car --> car
supplementary --> supplementari
real --> real
get --> get
purchase --> purchas
residential --> residenti
housing --> hous
ceremony --> ceremoni
second-hand --> second-hand
higher --> higher
transaction --> transact
renting --> rent
getting --> get
buying --> buy
educated --> educ
profile --> profil
building --> build
become --> becom
family --> famili
house --> hous
property --> properti
education --> educ
buy --> buy
construction --> construct
wedding --> wed
car --> car


From this list we can find the main words of each purpose that will help us to categorize the column.

In [23]:
#the list of the main stemmed words for each purpose    
real_estate = ['estat', 'hous', 'properti']   
wedding = ['wed']
education = ['educ','univers']
car = ['car']

In [24]:
#function for categorizing the purpose column
def group_by_purpose(purpose):
    stemmed_purpose = []
    for word in purpose.split(' '):
        stemmed_word = english_stemmer.stem(word)
        stemmed_purpose.append(stemmed_word)
        
    #check if any word in the purpose is in the key values of the purpose categories
    if any(item in stemmed_purpose for item in real_estate):
        return 'real estate'
    elif any(item in stemmed_purpose for item in wedding):    
        return 'wedding'
    elif any(item in stemmed_purpose for item in education):    
        return 'education'
    elif any(item in stemmed_purpose for item in car):    
        return 'car'
    return 'N/A'

#apply the function on the 'purpose' column
df['purpose_group'] = df['purpose'].apply(group_by_purpose)
#check there's no 'N/A' values 
display(df['purpose_group'].unique() ,df[['purpose','purpose_group']].head())

array(['real estate', 'car', 'education', 'wedding'], dtype=object)

,purpose,purpose_group
0,purchase of the house,real estate
1,car purchase,car
2,purchase of the house,real estate
3,supplementary education,education
4,to have a wedding,wedding


We succeded in categorizing the `purpose` column, now we'll check the distribution of the income so we'll be able to categorize the `total_income` into income levels:

In [25]:
df['total_income'].describe()

count     21452.000000
mean      26474.475573
std       15745.102943
min        3306.000000
25%       17199.000000
50%       23181.000000
75%       31501.000000
max      362496.000000
Name: total_income, dtype: float64

We'll devide the income level to 5 levels:
low, below average, average, above average, high

In [26]:
def group_by_income(income):
    if income < 15000:
        return 'low'
    elif income < 20000:
        return 'below average'
    elif income < 25000:
        return 'average'
    elif income < 30000:
        return 'above average'
    return 'high'

#apply the function on the 'income' column
df['income_level'] = df['total_income'].apply(group_by_income)
#check if changed 
df[['total_income','income_level']].head()

,total_income,income_level
0,40620,high
1,17932,below average
2,23341,average
3,42820,high
4,25378,above average


Now will categorize the `children` column, if a person has at least one kid it indicates True.

In [27]:
def group_by_children(children):
    if children != 0:
        return True
    return False

#apply the function on the 'income' column
df['has_kids'] = df['children'].apply(group_by_children)
#check if changed 
df[['children','has_kids']].head()

,children,has_kids
0,1,True
1,1,True
2,0,False
3,3,True
4,0,False


It can help to group the records by age, maybe can add some valuble conclusions.

In [28]:
def group_by_age(age):
    if age < 18:
        return 'N/A'
    elif age < 30:
        return '18-29'
    elif age < 60:
        return '30-59'
    return '60+'

#apply the function on the 'age' column
df['age_group'] = df['age'].apply(group_by_age)
#check if changed 
df['age_group'].unique()

array(['30-59', '18-29', '60+', 'N/A'], dtype=object)

#### Conclusion

Categorizing will help us to filter the data as desired to make our conclusions.

## Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [29]:
#creat a pivot table to show the relation between having kids and repaying a loan on time
having_kids = pd.pivot_table(df, index='has_kids', columns='debt', values='children', aggfunc='count', margins=True).reset_index()
having_kids['default_rate'] = (having_kids[True]/having_kids['All'])*100
having_kids2 = pd.pivot_table(df, index='children', columns='debt', values='has_kids', aggfunc='count', margins=True).reset_index()
having_kids2['default_rate'] = (having_kids2[True]/having_kids2['All'])*100
display(having_kids,having_kids2)

debt,has_kids,False,True,All,default_rate
0,False,13026,1063,14089,7.544893
1,True,6685,678,7363,9.208203
2,All,19711,1741,21452,8.115793


debt,children,False,True,All,default_rate
0,0,13026.0,1063.0,14089,7.544893
1,1,4410.0,445.0,4855,9.165808
2,2,1926.0,202.0,2128,9.492481
3,3,303.0,27.0,330,8.181818
4,4,37.0,4.0,41,9.756098
5,5,9.0,NaN,9,NaN
6,All,19711.0,1741.0,21452,8.115793


### Conclusion

From the above tables we can infer that people with kids are more likely to default on a loan than people without kids. People with four kids are the most likely to default on a loan, but there are only 41 records so maybe additional information will help. Parents usually have much more expenses and they want to fulfil their children's needs, that might affect their ability to repay their debts on time.

- Is there a relation between marital status and repaying a loan on time?

In [30]:
#creat a pivot table to show the relation between marital status and repaying a loan on time
marital_status = pd.pivot_table(df, index='family_status', columns='debt', values='family_status_id', aggfunc='count', margins=True).reset_index()
marital_status['default_rate'] = (marital_status[True]/marital_status['All'])*100
#check if there's a tendecy by age 
marital_status2 = pd.pivot_table(df[df['family_status'] == 'unmarried' ], index='age_group', columns='debt', values='family_status_id', aggfunc='count', margins=True).reset_index()
marital_status2['default_rate'] = (marital_status2[True]/marital_status2['All'])*100
display(marital_status, marital_status2)

debt,family_status,False,True,All,default_rate
0,civil partnership,3762,388,4150,9.349398
1,divorced,1110,85,1195,7.112971
2,married,11407,931,12338,7.545793
3,unmarried,2536,274,2810,9.750890
4,widow / widower,896,63,959,6.569343
5,All,19711,1741,21452,8.115793


debt,age_group,False,True,All,default_rate
0,18-29,814,103,917,11.232279
1,30-59,1476,161,1637,9.835064
2,60+,231,9,240,3.750000
3,N/A,15,1,16,6.250000
4,All,2536,274,2810,9.750890


### Conclusion

Unmarried are the most likely to default on a loan while widow / widower are the most likely to pay on time. We also checked the distribution by age groups for unmarried. It seems that the yonger you are the most likely you won't pay your debts on time. Many young people do not know how to manage themselves financially in their first years as independents.

- Is there a relation between income level and repaying a loan on time?

In [31]:
income_level = pd.pivot_table(df, index='income_level', columns='debt', values='total_income', aggfunc='count', margins=True).reset_index()
income_level['default_rate'] = (income_level[True]/income_level['All'])*100
#check if there's a tendecy by age 
income_level2 = pd.pivot_table(df[df['income_level'] == 'above average'], index='age_group', columns='debt', values='total_income', aggfunc='count', margins=True).reset_index()
income_level2['default_rate'] = (income_level2[True]/income_level2['All'])*100
display(income_level, income_level2)

debt,income_level,False,True,All,default_rate
0,above average,2891,294,3185,9.230769
1,average,3571,326,3897,8.365409
2,below average,4027,374,4401,8.498069
3,high,5775,449,6224,7.214010
4,low,3447,298,3745,7.957276
5,All,19711,1741,21452,8.115793


debt,age_group,False,True,All,default_rate
0,18-29,470,62,532,11.654135
1,30-59,2118,216,2334,9.254499
2,60+,289,14,303,4.620462
3,N/A,14,2,16,12.500000
4,All,2891,294,3185,9.230769


### Conclusion

People with around the average income are the most likely to default on a loan. Here also the youngest are at the top. The higher your income, the higher the chance you'll won't be in debt.

- How do different loan purposes affect on-time repayment of the loan?

In [32]:
loan_purpose = pd.pivot_table(df, index='purpose_group', columns='debt', values='purpose', aggfunc='count', margins=True).reset_index()
loan_purpose['default_rate'] = (loan_purpose[True]/loan_purpose['All'])*100
#check if there's a tendecy by age 
loan_purpose2 = pd.pivot_table(df[df['purpose_group'] == 'car' ], index='age_group', columns='debt', values='purpose', aggfunc='count', margins=True).reset_index()
loan_purpose2['default_rate'] = (loan_purpose2[True]/loan_purpose2['All'])*100
display(loan_purpose, loan_purpose2)

debt,purpose_group,False,True,All,default_rate
0,car,3903,403,4306,9.359034
1,education,3643,370,4013,9.220035
2,real estate,10027,782,10809,7.234712
3,wedding,2138,186,2324,8.003442
4,All,19711,1741,21452,8.115793


debt,age_group,False,True,All,default_rate
0,18-29,523,76,599,12.687813
1,30-59,2870,292,3162,9.234662
2,60+,491,32,523,6.118547
3,N/A,19,3,22,13.636364
4,All,3903,403,4306,9.359034


### Conclusion

People that take a loan for buying a car are the most likely to default on a loan. Also here, the youngest at the top with much higher rate.

## General conclusion

There are some parameters that may affect a potential borrower to repay their debts on time. If a person has kids or if his income is about the average, the more likely he will default the loan. Younger people and borrowers for buying a car are also at risk of being in debt.
Some of the criteria are distinct and some with a slight differences. It's better to collect more information about the customers so the conclusions will be consistent and bold in every report. There can be other influences on the conclusions like a major changes in the economy (affected by historical events such as an epidemic) so it will be better to add information about the date the loan was taken. 
There is also lack of clarity about data calculations and it is important to keep track of how it is stored in the databases.